In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon

import geopandas as gpd

import folium

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

In [2]:
from pyproj import Transformer

# WGS84 (lat/lon) <-> Web Mercator (meters)
to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
to_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def project_to_3857(lon, lat):
    return to_3857.transform(lon, lat)

def project_to_4326(x, y):
    return to_4326.transform(x, y)

In [3]:
def bezier_curve(start, control1, control2, end, n=100):
    """Generate a cubic Bezier curve."""
    t = np.linspace(0, 1, n)[:, None]  # shape (n, 1)
    start, control1, control2, end = map(np.array, [start, control1, control2, end])
    curve = (
        (1 - t)**3 * start
        + 3 * (1 - t)**2 * t * control1
        + 3 * (1 - t) * t**2 * control2
        + t**3 * end
    )
    return curve


In [4]:
import numpy as np
from shapely.geometry import Polygon

def unit(v):
    return v / np.linalg.norm(v)

def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left

def flow_patch_geo(
    start_ll, end_ll,
    start_width, end_width,
    bend=0.2,             # 0 = straight line, 0.3 = nice curve
    curvature=150,      # meters of sideways bend
):

    # ---- Project lon/lat to meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector
    d = end - start
    ud = unit(d)

    # Perpendicular
    n = perp(ud)

    # Control point positions:
    ctrl1 = start + d * bend + n * curvature
    ctrl2 = end   - d * bend + n * curvature

    # Build Bezier edges
    top = bezier_curve(
        start + n * (start_width/2),
        ctrl1 + n * (start_width/2),
        ctrl2 + n * (end_width/2),
        end   + n * (end_width/2)
    )

    bottom = bezier_curve(
        end   - n * (end_width/2),
        ctrl2 - n * (end_width/2),
        ctrl1 - n * (start_width/2),
        start - n * (start_width/2)
    )

    # Combine vertices
    verts = np.vstack([top, bottom, top[0]])

    # Back to lon/lat
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    poly = Polygon(verts_ll)
    return poly

In [28]:
E_line_polygons = []

Starting_Y_Val = 47.683729430644014
Second_Y_Val = 47.687259045590366
Third_Y_Val = 47.69108874059965

offset_const = 0.0005


### going to first stop
##main line
E_line_polygons.append(flow_patch_geo(
    start_ll = (-122.34433918947462, Starting_Y_Val),
    end_ll = (-122.34435249866571, Second_Y_Val),
    start_width = 150,
    end_width = 150,
    bend = 0,
    curvature = 0
))

## boarding line
E_line_polygons.append(flow_patch_geo(
    start_ll = (-122.34433918947462 - offset_const * 4, Starting_Y_Val),
    end_ll = (-122.34435249866571 - offset_const, Second_Y_Val),
    start_width = 20,
    end_width = 20,
    bend = 0,
    curvature = -50
))

### going to second stop

#main line
E_line_polygons.append(flow_patch_geo(
    start_ll = (-122.34435249866571, Second_Y_Val),
    end_ll = (-122.34437859849116, Third_Y_Val),
    start_width = 110,
    end_width = 110,
    bend = 0,
    curvature = 0
))

In [29]:
m = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=13)

for poly in E_line_polygons:
    folium.GeoJson(
        poly,
        style_function=lambda _,: {
            #"fillColor": 'red',
            "color": 'red',
            "weight": 1,
            "fillOpacity": 0.7
        }
    ).add_to(m)

m